# 원, 삼각형, 사각형 구분하기

In [1]:
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.models import Sequential
#층을 담는 그릇(Sequential은 keras에 포함되어 있는데
# 딥러닝 모델을 한층 한층 쌓기 쉽게 해주는 함수)
from tensorflow.keras.layers import Dense
#층을 만드는 것(각 층이 가질 특성을 각각 다르게 지정 가능) 
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")

In [2]:
# seed값 생성
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

## 데이터셋 생성하기

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1/255.)
# 데이터 값을 0과 1사이로 만듦
train_generator = train_datagen.flow_from_directory(
    'handwriting_shape/train',
    target_size=(24,24),
    batch_size=3,
    class_mode='categorical'
)

Found 45 images belonging to 3 classes.


In [7]:
test_datagen = ImageDataGenerator(rescale=1/255.)
# 데이터 값을 0과 1사이로 만듦
test_generator = test_datagen.flow_from_directory(
    'handwriting_shape/test',
    target_size=(24,24),
    batch_size=3,
    class_mode='categorical'
)

Found 15 images belonging to 3 classes.


In [8]:
train_generator.labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2])

In [9]:
train_generator.filenames[0]

'circle\\circle001.png'

## 모델 정의/설정/학습

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [11]:
model = Sequential([
    Conv2D(32, (3,3), input_shape=(24,24,3), activation='relu'),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 22, 22, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 10, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6400)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               819328    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 387       
Total params: 839,107
Trainable params: 839,107
Non-trainable params: 0
__________________________________________________

In [12]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics='accuracy'
)

## 모델 학습
케라스에서는 모델을 학습시킬 때 주로 fit() 함수를 사용하지만 제네레이터로 생성된 배치로 학습시킬 경우에는 fit_generator() 함수를 사용합니다. 본 예제에서는 ImageDataGenerator라는 제네레이터로 이미지를 담고 있는 배치로 학습시키기 때문에 fit_generator() 함수를 사용하겠습니다.

### 첫번째 인자 : 훈련데이터셋을 제공할 제네레이터를 지정합니다. 본 예제에서는 앞서 생성한 train_generator으로 지정합니다.
### steps_per_epoch : 한 epoch에 사용한 스텝 수를 지정합니다. 총 45개의 훈련 샘플이 있고 배치사이즈가 3이므로 15 스텝으로 지정합니다.
### epochs : 전체 훈련 데이터셋에 대해 학습 반복 횟수를 지정합니다. 100번을 반복적으로 학습시켜 보겠습니다.
### validation_data : 검증데이터셋을 제공할 제네레이터를 지정합니다. 본 예제에서는 앞서 생성한 validation_generator으로 지정합니다.
### validation_steps : 한 epoch 종료 시 마다 검증할 때 사용되는 검증 스텝 수를 지정합니다. 홍 15개의 검증 샘플이 있고 배치사이즈가 3이므로 5 스텝으로 지정합니다.

In [13]:
model.fit_generator(
        train_generator,
        steps_per_epoch=15,
        epochs=200,
        validation_data=test_generator,
        validation_steps=5)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/200
15/15 [==============================] - 0s 28ms/step - loss: 0.9845 - accuracy: 0.5333 - val_loss: 0.5359 - val_accuracy: 0.6667
Epoch 2/200
15/15 [==============================] - 0s 10ms/step - loss: 0.2971 - accuracy: 0.8889 - val_loss: 0.1873 - val_accuracy: 0.9333
Epoch 3/200
15/15 [==============================] - 0s 10ms/step - loss: 0.0368 - accuracy: 1.0000 - val_loss: 0.0821 - val_accuracy: 0.9333
Epoch 4/200
15/15 [==============================] - 0s 10ms/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.0816 - val_accuracy: 0.9333
Epoch 5/200
15/15 [==============================] - 0s 10ms/step - loss: 7.8768e-04 - accuracy: 1.0000 - val_loss: 0.0259 - val_accuracy: 1.0000
Epoch 6/200
15/15 [==============================] - 0s 10ms/step - loss: 4.1493e-04 - accuracy: 1.0000 - val_loss: 0.0210 - val_accuracy: 1.0000
Epoch 7/200
15/15 [==============================] - 0s 10ms/ste

## 모델 평가

In [14]:
model.evaluate_generator(test_generator, steps=5)

Instructions for updating:
Please use Model.evaluate, which supports generators.


[0.09764809161424637, 0.9333333373069763]

In [17]:
model.evaluate(test_generator, steps=5)

5/5 [==============================] - 0s 3ms/step - loss: 0.0976 - accuracy: 0.9333


[0.09764809161424637, 0.9333333373069763]